In [1]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]
    st = [(max(0, state[i*4+0] + state[i*4+2] * 4), abs(state[i*4+1] + state[i*4+3] * 4), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
#     print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [3]:

q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format(4)
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[ 0.05369189  0.0490973   0.05350574  0.07450765]
 [ 0.06754682  0.06143205  0.11508301  0.02168455]
 [ 0.04717754  0.01434293  0.0243172  -0.16420809]
 ...
 [ 0.30983893  0.44412759  0.61924409  0.22677398]
 [ 0.03884581 -0.23364447 -0.98649731 -0.57786193]
 [-0.63078854 -0.55038202 -0.26166878 -0.04381467]]


In [5]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action) 
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0
        
print("Training finished.\n")

Episode: 100
average reward: 32.93
Episode: 200
average reward: 35.8
Episode: 300
average reward: 38.71
Episode: 400
average reward: 21.11
Episode: 500
average reward: 60.16
Episode: 600
average reward: 59.57
Episode: 700
average reward: -2.69
Episode: 800
average reward: -9.15
Episode: 900
average reward: -39.42
Episode: 1000
average reward: 46.69
Episode: 1100
average reward: 70.41
Episode: 1200
average reward: 119.66
Episode: 1300
average reward: 31.6
Episode: 1400
average reward: 69.34
Episode: 1500
average reward: 112.29
Episode: 1600
average reward: 66.16
Episode: 1700
average reward: 42.05
Episode: 1800
average reward: 27.15
Episode: 1900
average reward: 51.91
Episode: 2000
average reward: 38.4
Episode: 2100
average reward: 67.38
Episode: 2200
average reward: 112.26
Episode: 2300
average reward: 39.72
Episode: 2400
average reward: -20.99


KeyboardInterrupt: 

In [6]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)

In [7]:
import time
total_epochs, total_penalties = 0, 0
episodes = 100
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 100 episodes:
Average timesteps per episode: 10801.0
Average penalties per episode: 107.45
Average reward per episode: 43.34


In [9]:
with open("weight.coe", 'w') as f:
    f.write("memory_initialization_radix=10;\n")
    f.write("memory_initialization_vector=\n")
    for i in range(3**10):
        f.write(str(np.argmax(q_table[i])))
        if(i == 3**10 - 1):
            f.write(";\n")
        else:
            f.write(",\n")